<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Pivot and Melt Introduction

___
<br>

This lesson is all about **transforming data** using `pandas`. Data transformation is the reorganization of your data set's rows and columns into a different, potentially **more useful shape and format**. 

The benefits of transforming your data include **better access to relevant information** and **streamlined data manipulation**. As you become more familiar with data sets and their associated operations, you will develop an intuition and appreciation for when it's better to **work row-wise or column-wise**.

Different data formats are better for different tasks. It takes time and experience to learn the distinctions. But, for now, we'll introduce the **common structures, transformations, and how to apply these transformations**.

### Learning Objectives
- Understand the differences between **long and wide format data**.
- Understand **pivot tables**.
- Practice transforming data between **long and wide** formats.
- Practice creating pivot tables.
- Learn how to avoid **common pitfalls and obstacles** in data transformation with `pandas`.

### Lesson Guide

- [Wide Format Data](#wide_format)
- [Melting Wide Format to Long Format](#melt_data)
- [Pivot Long Format to Wide Format](#pivot_data)
- [MultiIndex/Hierarchical Indices in `pandas`](#multiindex)
- [Summarizing Data With `.pivot_table()` and Aggregate  Functions](#pivot_table_summarizing)
- [Getting Rid of the MultiIndex: "Flattening" Data](#multiindex_to_flat)


In [ ]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)

sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline


<a id='wide_format'></a>

### Wide Format Data

---

Between "wide" and "long," **wide format data is the more intuitive**. It's also a common format for `.csv` files. You've already viewed multiple data sets in wide format throughout this course.

Wide format data is structured so that:

- Unique IDs, subjects, observations, etc. are represented as **rows**.
- Distinct information categories (**variables**) are represented as columns. In other words, there is a **column for every "variable"** with its own unique values.
- This format can often be a more compact matrix, particularly if little or no information is missing.
- It is **not as useful for SQL-style operations**: It can make it much harder or even impossible to **join tables together on a value**.
- It can be useful in `pandas` when you need to perform operations on variables **across columns**; for example, multiplying columns together to create a new column.
- It is the data format required for statistical modeling (with few exceptions).

Here is a simple example of wide data, where each row represents one student's scores in different subjects.

In [ ]:
df_wide = pd.read_csv('datasets/students_wide.csv')
df_wide

How might you want to analyse the data in the DataFrame above? What would you need to do?

<a id='melt_data'></a>

### Melting Wide to Long

The `pandas` `melt()` function can be used to transform a wide format dataframe into a long format, where a new column `variable` represents the column and the `value` is the value stored in that column for a particular unique ID.

In [ ]:
# Melt the df_wide dataframe into long format by each student_ID
df_long1 = pd.melt(df_wide, id_vars='Student ID')
df_long1

In [ ]:
# Change the column headers to represent the original meaning
df_long1 = pd.melt(df_wide, id_vars='Student ID', var_name='Subject', value_name = 'Score')
df_long1

### Using `melt` on DataFrame directly

We can also call the `melt` method on the DataFrame, and specify the column names to use for the variable and value respectively:

In [ ]:
# Using melt method on df_wide
df_long2 = df_wide.melt(id_vars='Student ID', var_name='Subject', value_name = 'Score')
df_long2

## Specifying identifying rows

We can specify which columns are to be the identifiers. Let's say we add another column to represent the student's name.

In [ ]:
df_wide['Major']=pd.Series(['Arts','Arts','Science','Science','Science'])
df_wide

Now we can include `Major` as an identifier variable along with the `Student ID`, for the variable `Subject` with value `Score`

In [ ]:
df_long3=df_wide.melt(id_vars=['Student ID','Major'], var_name='Subject', value_name='Score')
df_long3

## Missing values

In all the above we have many `NaN` values because the values were missing in the original wide format. In this case we can just drop the nulls, as they are not necessary in the long format. 


In [ ]:
df_long3.dropna(inplace=True)
df_long3

### Save the long format data

Now that the data is in long format, we can save it.



In [ ]:
df_long3.to_csv('datasets/df_long.csv')

<a id='pivot_data'></a>

## Pivot to Transform from Long to Wide

The `pivot` function is used to transform from long to wide format. 
Let's use the file that we have saved from above.


In [ ]:
df_long = pd.read_csv('datasets/df_long.csv')

In [ ]:
df_wide1 = pd.pivot(df_long, index=['Student ID','Major'], columns='Subject', values='Score')
df_wide1

<a id='multiindex'></a>


### MultiIndex 
In the header, you can see that the format of the new wide data **is *not* the same as our originally loaded wide format**. `pandas` implements something called **MultiIndexing** or **hierarchical indexing**, which allows for "tiered" row and column labels.

The main difference is that we now have a `variable` name in the top left corner, which is **"labeling"** our columns (and corresponds to the name of our original column in the long format data). The row indexer has become our **single key/ID variable**, `Student ID` and `Major`. The columns are what we would expect here: **Each one is a variable**, like in the original wide format data.

We can reset the index to the default row numbering starting from 0, which will set the `Student ID` and `Major` as  columns again.

In [ ]:
df_wide1 = df_wide1.reset_index()
df_wide1

The variable name `Subject` is now used to name the columns, as we can see if we list all the columns of the DataFrame.

In [ ]:
df_wide1.columns

We can reset the name to `None` and this will put make the format what we are used to.

In [ ]:
df_wide1.columns.name=None
df_wide1

### Using pivot() with the DataFrame

We can also use the `pivot()` method with the DataFrame. Let's reset the index in the same step, and set the column name to `None` too. 

In [ ]:
df_wide2 = df_long3.pivot(index=['Student ID','Major'], columns='Subject', values='Score').reset_index()
df_wide2.columns.name=None

In [ ]:
df_wide2

<a id='pivot_table_summarizing'></a>

### Summarizing Your Data With  `.pivot_table()` and Aggregate Functions

---
We have used `pivot` to transform long format data to wide format. A related function `pivot_table()` can also perform the same transformation, but you can also summarize the values using an aggregate function such as finding the mean, count, sum, just like Excel pivot tables. 

Below, I am calling the `.pivot_table()` function with:

- The long format data as the first argument.
- `Subject` specified as the columns that indicate the variable names (groups).
- `Score` specified as the column that contains the data per variable.
- `Major` as the index; the rows will be grouped by `Major`.
- `np.mean`, `np.median`, `np.std`, and `len` as aggregate functions. These will be calculated for each `Major-by-Sariable` group.
- A `fill_value` of `np.nan` for cells in the output table that have no data.

In [ ]:
df_wide_pv = pd.pivot_table(df_long3, columns=['Subject'], values='Score',
                                     index=['Major'], aggfunc=[np.mean, np.median, np.std, len],
                                     fill_value=np.nan)
df_wide_pv

The output DataFrame gives you a "hierarchical" column index — the three variables for each aggregate function. The row index is the `major` groups.

If you apply more index variables, the row indices will also become hierarchical! However, this can quickly make for a bloated DataFrame.

In [ ]:
df_wide_pv.columns

<a id='multiindex_to_flat'></a>

### Getting Rid of the MultiIndex: "Flattening" Data

---

The most reliable way to "flatten" a MultiIndexed DataFrame is with the `.to_records()` function. To make this a new DataFrame, it needs to be wrapped in a `pd.DataFrame()` like so:

In [ ]:
results_summary = pd.DataFrame(df_wide_pv.to_records())
results_summary.head(2)

You can see that the new column names are tuples of the hierarchy of MultiIndexed columns. For example, you could convert these to new, more easily indexed columns with something like a list comprehension.

We can use the `join()` and `.eval()` functions to combine the tuple as a single String, for example:


In [ ]:
'_'.join(eval("('mean','Biology')"))

**Use a list comprehension and the `eval()` and `join()` functions to convert the flattened MultiIndexed columns to something more readable.**

In [ ]:
results_summary.columns

In [ ]:
results_summary.columns = ['_'.join(eval(c)) if c[0]=='(' else c for c in results_summary.columns]

In [ ]:
results_summary